In [192]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
from beakerx import *
from beakerx.object import beakerx
import scipy.stats as stats
from copy import copy
from IPython.display import HTML

In [193]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# A Sample of Systematic Trading Strategies

The goal of this file is to provide an easy way to visualize the returns to a variety of systematic trading strategies, across asset classes and across countries. Factors include:

1. Classic cross sectional equity factors: value, momentum, betting against beta and quality. 
2. Value, momentum, and betting against beta in other asset classes. 
3. Time series momentum, which differs from the cross sectional strategies above in that it is not market neutral at all points in time.

Two facts jump out to me when I look at these charts.

1. Factor strategies no longer earn the returns they used to. Since 2008, the returns to a large set of "robust" textbook factors that span asset classes and countries have gone down by roughly 50%. Since these factors were well known, robust factors, I interpret the decay in returns as evidence that funds are getting better at trading on systematic trading strategies over time and that to the extent these factors represented risk factors, these risks are becoming more widely held. 
2. There's no way that these factors represent "tradable returns" accessible to an investor. A diversified basket of all the equity factors across all the countries earns a Sharpe ratio of 1.6, while being uncorrelated to the broader market. 

In [194]:
# Parameters
CUTOFF_DATE = pd.to_datetime('2008-01-01', format = '%Y-%m-%d') + MonthEnd(0)
START_DATE = '1930-01-01'
NORM_AVG = 0.10 / 12
NORM_STD = 0.10 / np.sqrt(12)
ALL_COUNTRIES = ['USA', 'WLD', 'GBR', 'JPN', 'DEU', 'FRA', 'ITA', 'CAN', 'AUS']
ALL_CTRY_EX_WORLD = copy(ALL_COUNTRIES)
ALL_CTRY_EX_WORLD.remove('WLD')
ALL_ASSETS = ['Equities', 'FX', 'Bonds', 'Cmd']

# Import data
factors_raw = read_hdf('../Data/data.h5', key = 'all_factors')

def normalize_factors(factor_dataset, normalization_var, aggregation_level, country_list, cutoff_date = CUTOFF_DATE, start_date = START_DATE):
    factors = factor_dataset.loc[factor_dataset.index.get_level_values('Country').isin(country_list), :]
    factors = factors.groupby(aggregation_level + ['Month_date']).mean()
    factors = factors.reset_index().set_index(aggregation_level)
    factors = factors.loc[factors['Month_date'] >= start_date, :]
    
    # Build return variables
    factors['Log Return'] = np.log(factors['Month_ret'] + 1)
    factors['Before Cutoff'] = (factors['Month_date'] < cutoff_date)
    factors = factors.reset_index().set_index(aggregation_level)
    if normalization_var == 'Avg':
        factors['Avg Return Before Cutoff'] = factors.groupby(by = aggregation_level).apply(lambda d: d.loc[d['Before Cutoff'], 'Log Return'].mean())
        factors['Normalized Log Return'] = factors['Log Return'] * NORM_AVG / factors['Avg Return Before Cutoff']
    elif normalization_var == 'Vol':
        factors['Avg Vol Before Cutoff'] = factors.groupby(by = aggregation_level).apply(lambda d: d.loc[d['Before Cutoff'], 'Log Return'].std())
        factors['Normalized Log Return'] = factors['Log Return'] * NORM_STD / factors['Avg Vol Before Cutoff']
    else:
        assert(0)
        
    factors['Normalized Cumulative Return'] = factors.groupby(by = aggregation_level)['Normalized Log Return'].cumsum()
    factors['Cumulative Return on Cutoff'] = factors.groupby(by = aggregation_level).apply(lambda d: d.loc[d['Month_date'] == cutoff_date, 'Normalized Cumulative Return'].values[0])
    factors['Normalized Cumulative Return'] = factors['Normalized Cumulative Return'] - factors['Cumulative Return on Cutoff']
    return factors

# CMD Value looks particularly bad
factors_to_use = factors_raw
factors_to_use = factors_to_use.loc[~((factors_to_use.index.get_level_values('Asset Class') == 'Cmd') &\
                                      (factors_to_use.index.get_level_values('Descriptor') == 'Value')), :]
factors_to_use = factors_to_use.reset_index().set_index(['Descriptor', 'Asset Class', 'Country'])

# Strategies Averaged Across Countries

In [195]:
def build_plot_for_frame(factor_dataset, plot_title = 'Cumulative Returns', **kwargs):

    all_types = factor_dataset.index.unique().values
    cum_ret_plots = TimePlot(title = plot_title, legendLayout=LegendLayout.HORIZONTAL,\
                              legendPosition=LegendPosition(position=LegendPosition.Position.TOP), **kwargs)

    for x in all_types:
        if type(x).__name__ == 'str':
            string_name = x
        else:
            string_name = ' '.join(x)            
        cum_ret_plots.add(Line(displayName = string_name, \
                               x = factor_dataset.loc[x, 'Month_date'],\
                               y = factor_dataset.loc[x, 'Normalized Cumulative Return']))
    return cum_ret_plots

In [196]:
asset_class_by_strategy = normalize_factors(factors_to_use, 'Avg', ['Asset Class', 'Descriptor'], ALL_COUNTRIES, start_date = '1960-01-01')

## Summary Statistics

In [197]:
summary = asset_class_by_strategy.groupby(['Asset Class', 'Descriptor'])['Month_ret'].describe()

In [198]:
summary

In [199]:
build_plot_for_frame(asset_class_by_strategy, plot_title = 'Cumulative Returns (ln, Normalized to 10% Avg Return)', initWidth = 1000, initHeight = 800)

# Country Level Deep Dive

In [200]:
country_equities = normalize_factors(factors_to_use, 'Vol', ['Asset Class', 'Descriptor', 'Country'], ALL_CTRY_EX_WORLD, start_date = '1988-01-01')

In [201]:
ctry_plots = []
for c in ALL_CTRY_EX_WORLD:
    ctry_plots.append(build_plot_for_frame(country_equities.loc[country_equities.index.get_level_values('Country') == c, :], plot_title = 'Cum. Ret (ln, Norm. to 10% Vol)', initWidth = 500, initHeight = 400))

In [202]:
def plot_list(list_of_beakerx_plots):

    lg = GridOutputContainerLayoutManager(3)
    og = OutputContainer()
    og.setLayoutManager(lg)
        
    for p in list_of_beakerx_plots:
        og.addItem(p)
    return og

In [203]:
plot_list(ctry_plots)

GridView(children=(BeakerxHBox(children=(TimePlot(model={'chart_title': 'Cum. Ret (ln, Norm. to 10% Vol)', 'co…

# Returns to different classes of strategies

In [204]:
returns_by_strategy = normalize_factors(factors_to_use, 'Avg', ['Descriptor'], ALL_COUNTRIES, start_date = '1930-01-01')

In [205]:
build_plot_for_frame(returns_by_strategy, plot_title = 'Cumulative Returns (ln, Normalized to 10% Average)', initWidth = 1000, initHeight = 700)

# Returns to different asset classes

In [206]:
returns_by_asset_class = normalize_factors(factors_to_use, 'Vol', ['Asset Class'], ALL_COUNTRIES, start_date = '1970-01-01')
build_plot_for_frame(returns_by_asset_class, plot_title = 'Cumulative Returns (ln, Normalized to 10% Vol)', initWidth = 1000, initHeight = 700)

## A Formal Statistical Test

Suppose the null hypothesis is that there is no structural break. Suppose that return errors are essentially uncorrelated across time but correlated across strategies. Thus

$$ R_t \sim N(\mu, \Sigma) \mbox{ iid Across Time}$$

Suppose that I take the sample mean in the first T periods and then another sample mean in the last S periods. We then have that the distribution of this difference in sample means is

$$ T^{-1} \Sigma_{t=1}^{t=T} R_t - S^{-1} \Sigma_{t=T+1}^{t=T+S} R_t \sim N\left(0, (T^{-1} + S^{-1}) \times \Sigma \right)$$

We can use any consistent estimator of this variance covariance matrix, so we can just take the full sample estimate. The chart below shows the p value of this test at different split dates. The test does not detect a structural break until somewhere in the 2003-2010 region.

In [207]:
wide_returns = asset_class_by_strategy[['Month_date', 'Log Return']].reset_index()
wide_returns['strategy'] = [x + '_' + y for x,y in zip(wide_returns['Asset Class'], wide_returns['Descriptor'])]
wide_returns = wide_returns[['Month_date', 'strategy', 'Log Return']].pivot('Month_date', 'strategy', 'Log Return')

Sigma = wide_returns.cov().values

def calc_test_stat(wide_returns, Sigma, split_date):
    first_data = wide_returns[wide_returns.index < split_date]
    second_data = wide_returns[wide_returns.index > split_date]
    
    CONSERVATIVE_T = first_data.dropna().shape[0]
    T = 0.3 * first_data.shape[0] + 0.7 * CONSERVATIVE_T
    S = second_data.shape[0]
    p = first_data.shape[1]
    
    if T * S == 0:
        return (np.nan, np.nan, np.nan, np.nan)
    
    x_bar_first = first_data.mean()
    x_bar_second = second_data.mean()
    variances = np.diag(Sigma)
    standard_deviations = np.sqrt(variances)
    jensen = variances / 2
    return_precision = np.linalg.inv(Sigma)
    
    # An optimal degradation measure based on a precisely measured covariance matrix
    pre_arith_returns = x_bar_first + variances / 2
    post_arith_returns = x_bar_second + variances / 2
    pre_optimal_sharpe = np.sqrt(pre_arith_returns.T  @ return_precision @ pre_arith_returns)
    post_optimal_sharpe = np.sqrt(post_arith_returns.T @ return_precision @ post_arith_returns)
    sr_degrade = post_optimal_sharpe / pre_optimal_sharpe
        
    # A more robust estimator
    init_sharpe_ratios = np.divide(x_bar_first.values + variances / 2, standard_deviations)
    later_sharpe_ratios = np.divide(x_bar_second.values + variances / 2, standard_deviations)
    ret_degrade = np.median(later_sharpe_ratios / init_sharpe_ratios)
    
    # Build the test statistic
    multiplier = (T ** -1 + S ** -1) ** -1
    ret_diff = x_bar_second - x_bar_first
    test_stat = multiplier * ret_diff.T @ return_precision @ ret_diff
    
    if np.isnan(test_stat):
        return (np.nan, np.nan, np.nan, np.nan) 
    
    return (float(stats.chi2.sf(test_stat, p)), sr_degrade, ret_degrade, test_stat)

test_results = [calc_test_stat(wide_returns, Sigma, x) for x in wide_returns.index]

In [208]:
p_seq = [x[0] for x in test_results]
sr_seq = [x[1] for x in test_results]
avg_sr_seq = [x[2] for x in test_results if len(x) > 0]

result_frame = pd.DataFrame.from_dict({'Month_date': wide_returns.index, 'P Values': p_seq, 'Optimal SR Degradation': sr_seq, 'Avg SR Degradation': avg_sr_seq})

result_frame = result_frame.loc[result_frame['Month_date'].between('1990-01-31', '2011-12-31')]

def simple_plot(dataframe, variable, plot_title, series_title):
    plot = TimePlot(title = plot_title, legendLayout=LegendLayout.HORIZONTAL,\
                          legendPosition=LegendPosition(position=LegendPosition.Position.RIGHT),\
                        initWidth = 1000,\
                        initHeight = 700)
    
    plot.add(Line(displayName = series_title, \
                       x = dataframe['Month_date'],\
                       y = dataframe[variable]))
    plot.setYBound(0, 1)
    return plot
    
simple_plot(result_frame, 'P Values', 'P Value of Test of Structural Break by Split Date', 'P Value')

Another way to visualize the breakdown in returns is to take the average Sharpe ratio in the "post" period across these strategies and divide it by the average Sharpe ratio in the "pre" period, splitting by various dates. This suggests that Sharpe ratios have fallen by around 60% (i.e. they are 40% of what they used to be) in the post 2008 period.

In [209]:
simple_plot(result_frame, 'Avg SR Degradation', 'Reduction in Average Sharpe Ratios by Split Date', 'Ratio')

## Future Work

1. Incorporate more factors?
2. Take out the unconditional loading on overall market risk factors such as bonds or equities. For example, time series momentum on bonds does really well in the post 2008 periods because it's been a 10 year bull market in bonds. Therefore the results above likely overstate the returns to some of these strategies.
3. Investigate the construction of some of these series? I think they're all flawed because they all use some variant on equal weighting -- i.e. that they require the fund to have large stakes in small companies. How would these series look if we did a different kind of replication?